In [1]:
import webdataset as wds



In [2]:
from training.common import prepare_datasets
from training.mast3r.train import DataConfig
from datasets import scene, chunk

data_config = DataConfig.load_from_files([
    "./config/data/base.yaml",
])

data_config.scenes = ["47b37eb6f9"]

base_dataset = scene.Dataset(data_config)
base_dataset.prepare_data()
image_dataset = chunk.image.Dataset(data_config, base_dataset)

mast3r_dataset = chunk.mast3r.Dataset(data_config, base_dataset, image_dataset)
mast3r_dataset.prepare_data()

100%|██████████| 1/1 [00:00<00:00, 18.95it/s]


In [3]:
mast3r_dataset[0]

{'scene_name': '47b37eb6f9',
 'file_name': 'DSC03662.JPG',
 'image_name_chunk': 'DSC03662.JPG',
 'pairwise_predictions': ({'pts3d': tensor([[[[-4.4116, -2.8132,  3.9073],
             [-4.5030, -2.9563,  3.9180],
             [-4.4908, -2.9761,  3.8983],
             ...,
             [ 4.5159, -2.9623,  3.9239],
             [ 4.4944, -2.9316,  3.8894],
             [ 4.4330, -2.8619,  3.8861]],
   
            [[-4.4892, -2.9588,  3.8656],
             [-4.5454, -2.9696,  3.9697],
             [-4.5117, -2.9980,  3.9300],
             ...,
             [ 4.5545, -2.9834,  3.9517],
             [ 4.4474, -2.9162,  3.8765],
             [ 4.5565, -2.9160,  3.9035]],
   
            [[-4.4716, -2.9437,  3.8322],
             [-4.5466, -2.9888,  3.9803],
             [-4.5006, -2.9557,  3.9371],
             ...,
             [ 4.5515, -2.9547,  3.9405],
             [ 4.5005, -2.9660,  3.9419],
             [ 4.5401, -2.9402,  3.9447]],
   
            ...,
   
            [[-1.0205,  0

In [15]:
# maxsize should be 10 gb
import tqdm


maxsize = 10 * 1024 * 1024 * 1024
with wds.ShardWriter("/mnt/dorta/mast3r/wids/db_%06d.tar", maxcount=100, maxsize=10e9) as writer:
    # load mast3r_dataset
    for i in tqdm.tqdm(range(len(mast3r_dataset))):
        loaded = mast3r_dataset[i]
        scene_name, file_name, image_name_chunk, pairwise_predictions, pairs_image_names = loaded["scene_name"], loaded["file_name"], loaded["image_name_chunk"], loaded["pairwise_predictions"], loaded["pairs_image_names"]
        object = {
            "__key__": f"item_{i}",
            "meta.pickle": {
                "scene_name": scene_name,
                "file_name": file_name,
                "image_name_chunk": image_name_chunk,

            "pairs_image_names": pairs_image_names,
            
            },
            **{
                f"predictions_{i}_{key}.pth": value
                for i in range(len(pairwise_predictions))
                for key, value in pairwise_predictions[i].items()
            },
        }
        writer.write(object)



# writing /mnt/dorta/mast3r/wids/db_000000.tar 0 0.0 GB 0


 21%|██▏       | 64/300 [00:25<02:10,  1.81it/s]

# writing /mnt/dorta/mast3r/wids/db_000001.tar 63 10.1 GB 63


 42%|████▏     | 127/300 [01:12<03:32,  1.23s/it]

# writing /mnt/dorta/mast3r/wids/db_000002.tar 63 10.1 GB 126


 63%|██████▎   | 190/300 [02:06<01:10,  1.56it/s]

# writing /mnt/dorta/mast3r/wids/db_000003.tar 63 10.1 GB 189


 84%|████████▍ | 253/300 [02:48<00:23,  2.03it/s]

# writing /mnt/dorta/mast3r/wids/db_000004.tar 63 10.1 GB 252


100%|██████████| 300/300 [03:28<00:00,  1.44it/s]


In [21]:
import glob
import os
from wids import ShardListDataset

total = 300
normal_size = 63

# get number of samples in file
def get_samples(i):
    # normally every file has 19 samples
    # only the last file has fewer samples to make up for the total number of samples
    if i == len(files) - 1:
        return total - normal_size * (len(files) - 1)
    return normal_size

files = glob.glob("/mnt/dorta/mast3r/wids/db_*.tar")
#   shards: a list of (filename, length) pai
shards = [{ "url": file, "nsamples": get_samples(i) } for i, file in enumerate(files)]

In [22]:
import wids

dataset = wids.ShardListDataset(shards=shards, base="/mnt/dorta/mast3r/wids/")

[{'url': '/mnt/dorta/mast3r/wids/db_000003.tar', ' base: None name: None nfiles: 5 nbytes: 0 samples: 300 cache: /tmp/_wids_cache


In [23]:
sampler = wids.ShardedSampler(dataset)

# collect the first 10 samples
for i in range(10):
    sample = next(iter(sampler))
    element = dataset[sample]
    print(element)
    


{'.meta.pickle': {'scene_name': '47b37eb6f9', 'file_name': 'DSC04017.JPG', 'image_name_chunk': 'DSC04017.JPG', 'pairs_image_names': [('DSC04017.JPG', 'DSC04030.JPG'), ('DSC04017.JPG', 'DSC04003.JPG'), ('DSC04017.JPG', 'DSC04032.JPG'), ('DSC04017.JPG', 'DSC04013.JPG')]}, '.predictions_0_conf.pth': tensor([[[1.1547, 1.1694, 1.1029,  ..., 2.5817, 2.5662, 3.5591],
         [1.1235, 1.1390, 1.1486,  ..., 2.5071, 2.1886, 2.5124],
         [1.1079, 1.1704, 1.0401,  ..., 2.5494, 2.3898, 2.5367],
         ...,
         [3.6877, 3.7951, 3.8465,  ..., 5.2772, 4.3062, 3.8980],
         [4.8287, 3.5118, 3.8310,  ..., 4.9240, 3.2463, 3.9875],
         [5.4977, 4.8841, 4.2647,  ..., 4.2680, 4.1344, 5.5512]],

        [[1.0276, 1.0169, 1.0084,  ..., 2.2402, 2.2485, 2.9137],
         [1.0126, 1.0110, 1.0038,  ..., 2.0629, 1.8400, 2.1353],
         [1.0067, 1.0038, 1.0009,  ..., 2.1582, 1.9959, 2.1420],
         ...,
         [2.3802, 2.3074, 2.6966,  ..., 3.2378, 2.6333, 2.3411],
         [2.7245, 2.09

In [5]:
import webdataset as wds

list = wds.shardlists.DirectoryShardList("/mnt/dorta/mast3r/wids/")

dataset = wds.DataPipeline(list)
next(iter(dataset))


KeyboardInterrupt: 

In [26]:
import wids

wids.ShardListDataset(shards=shards, base="/mnt/dorta/mast3r/wids/")

TypeError: tuple indices must be integers or slices, not str

In [1]:
# first load data_config
from training.mast3r.train import Config, DataConfig
from training.common import create_datamodule

data_config = DataConfig.load_from_files([
    "./config/data/base.yaml",
    "./config/data/images.yaml"
], {

    "force_prepare_mast3r": True,
})

config = Config.load_from_files([
    "./config/trainer/base.yaml",
    "./config/network/base_unet.yaml",
    "./config/network/unet3D.yaml",
    "./config/module/base.yaml",
], {
    **data_config.model_dump(),
    "in_channels": data_config.get_feature_channels(),
})


config.base_channels = 16
config.name = "mast3r-3d-experiments"
config.max_epochs = 1
config.prefetch_factor = 2
config.num_workers = 11
config.val_num_workers = 6

datamodule = create_datamodule(config, splits=["train", "val"])

2025-01-28 19:54:00.682 | INFO     | training.common:create_dataset:37 - Creating dataset for split train
  0%|          | 0/280 [00:00<?, ?it/s]

Mesh not found for scene 75d29d69b8. Skipping.
Mesh not found for scene 036bce3393. Skipping.
Mesh not found for scene 394a542a19. Skipping.
Mesh not found for scene 9859de300f. Skipping.Mesh not found for scene 076c822ecc. Skipping.



 43%|████▎     | 120/280 [00:00<00:00, 1119.17it/s]

Mesh not found for scene b09431c547. Skipping.


100%|██████████| 280/280 [00:00<00:00, 1165.30it/s]
2025-01-28 19:54:01.207 | INFO     | training.common:create_dataset:37 - Creating dataset for split val
  0%|          | 0/50 [00:00<?, ?it/s]

Mesh not found for scene 3864514494. Skipping.


100%|██████████| 50/50 [00:00<00:00, 1396.66it/s]


In [7]:
datamodule.prepare_data()

2025-01-28 19:54:04.897 | INFO     | datasets.chunk.zip:prepare_data:30 - Preparing dataset datasets.chunk.image.Dataset
2025-01-28 19:54:16.259 | INFO     | datasets.chunk.zip:prepare_data:30 - Preparing dataset datasets.chunk.occupancy_revised.Dataset
2025-01-28 19:54:32.871 | INFO     | datasets.chunk.zip:prepare_data:30 - Preparing dataset datasets.chunk.mast3r.Dataset


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Warning, cannot find cuda-compiled version of RoPE2D, using a slow pytorch version instead


In [ ]:
dataloader = datamodule.train_dataloader()

# time what a whole epoch looks like
import time
start = time.time()
for i, batch in enumerate(dataloader):
    if i > 10:
        break
end = time.time()
print(f"Time for 10 batches: {end - start}")
